To run this notebook using the project venv, you will need to install the jupyter into it. After activate your venv, run the commands below:

```bash
$ (venv) pip3 install jupyter
$ (venv) ipython kernel install --name "hyperheuristics-venv" --user
```

Next, just restart your Jupyter and change the kernel that your Jupyter notebook is using (if you are running it into VSCode, the kernel is located in the top right corner). 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import numpy as np
import hashlib
import yaml
import json
import copy
import os

from typing import Text, Dict

from src.hyperHeuristics import HyperHeuristic
from utils.generator.knapsackInstanceGenerator import generateKnapsackInstance
from src.operators import getAvailableOperators
from utils.knapsackSortFunctions import getAvailableSortIndexesFunctions

In [3]:
# Data path
DATA_PATH = "../data/raw"
os.makedirs(DATA_PATH, exist_ok = True)

In [4]:
config = yaml.load(open('experimentConfig.yaml'), yaml.Loader)
stages = config['stages']
firstStages = np.array(stages['firstStage'])
secondStages = np.array(stages['secondStage'])
thirdStages = np.array(stages['thirdStage'])
params = config['params']

In [5]:
def getRandomOperator(operator):
    operatorParams = params[operator]

    sortedParams = {param: values[int(np.random.rand() * len(values))] for param, values in operatorParams.items()}

    sortedParams['operatorName'] = operator

    return sortedParams

In [6]:
def getRandomStageParams(stage: Text) -> Dict:
    stageParams = params[stage]

    sortedParams = {param: values[int(np.random.rand() * len(values))] for param, values in stageParams.items()}

    if stage == 'alns':
        destroyOperators = params['alns']['destroyOperators']
        repairOperators = params['alns']['repairOperators']

        # Destroy operators
        sortedParams['destroyOperators'] = [getRandomOperator(operator) for operator in destroyOperators]

        # Repair operators
        sortedParams['repairOperators'] = [getRandomOperator(operator) for operator in destroyOperators]

    return sortedParams

In [7]:
def parseFunctionsAndObjectsInParams(stageName, stageParams):
    stageParams = copy.deepcopy(stageParams)

    if stageName == 'alns':
        # Destroy operators
        destroyOperators = stageParams['destroyOperators']
        for i, operatorParams in enumerate(destroyOperators):
            if 'sortIndexesFunction' in operatorParams:
                sortFunction = operatorParams['sortIndexesFunction']
                stageParams['destroyOperators'][i]['sortIndexesFunction'] = getAvailableSortIndexesFunctions()[sortFunction]
           
            operatorName = operatorParams['operatorName']
            stageParams['destroyOperators'][i] = getAvailableOperators()[operatorName](**operatorParams)

        # Repair operators
        repairOperators = stageParams['repairOperators']
        for i, operatorParams in enumerate(repairOperators):
            if 'sortIndexesFunction' in operatorParams:
                sortFunction = operatorParams['sortIndexesFunction']
                stageParams['repairOperators'][i]['sortIndexesFunction'] = getAvailableSortIndexesFunctions()[sortFunction]
           
            operatorName = operatorParams['operatorName']
            stageParams['repairOperators'][i] = getAvailableOperators()[operatorName](**operatorParams)

    elif stageName == 'greedyConstruction':
        sortFunction = stageParams['sortIndexesFunction']
        stageParams['sortIndexesFunction'] = getAvailableSortIndexesFunctions()[sortFunction]
        
    return stageParams

In [8]:
def getRandomThreeStagesPipeline(
        firstStageCandidates: np.array, 
        secondStageCandidates: np.array, 
        thirdStageCandidates: np.array
    ) -> HyperHeuristic:
    
    hyperheuristic = HyperHeuristic()
    lowLevelFunctions = hyperheuristic.getAvailableLowLevelHeuristics()

    # Sort first stage
    firstStage = np.random.choice(firstStageCandidates, 1)[0]
    firstStageParams = getRandomStageParams(firstStage)

    firstStageCandidates = firstStageCandidates[firstStageCandidates != firstStage]
    thirdStageCandidates = thirdStageCandidates[thirdStageCandidates != firstStage]

    # Sort second stage
    secondStage = np.random.choice(secondStageCandidates, 1)[0]
    secondStageParams = getRandomStageParams(secondStage)

    thirdStageCandidates = thirdStageCandidates[thirdStageCandidates != secondStage]

    # Sort third stage
    thirdStage = np.random.choice(thirdStageCandidates, 1)[0]
    thirdStageParams = getRandomStageParams(thirdStage)

    stages = [
        {
            "name": firstStage,
            "callback": lowLevelFunctions[firstStage],
            "inputSchema": ['problemInstance'],
            "outputSchema": ['problemInstance', 'solution'],
            "params": parseFunctionsAndObjectsInParams(firstStage, firstStageParams)
        },
        {
            "name": secondStage,
            "callback": lowLevelFunctions[secondStage],
            "inputSchema": ['problemInstance','solution'],
            "outputSchema": ['problemInstance', 'solution'],
            "params": parseFunctionsAndObjectsInParams(secondStage, secondStageParams)
        },
        {
            "name": thirdStage,
            "callback": lowLevelFunctions[thirdStage],
            "inputSchema": ['problemInstance','solution'],
            "outputSchema": ['problemInstance', 'solution'],
            "params": parseFunctionsAndObjectsInParams(thirdStage, thirdStageParams)
        },
    ]

    hyperheuristic.addHyperHeuristicComponentsFromDict(stages)

    print(" -> ".join(hyperheuristic.getStagesNames()))

    return hyperheuristic, {
        'firstStage': {'name':firstStage, 'params':firstStageParams},
        'secondStage': {'name':secondStage, 'params':secondStageParams},
        'thirdStage': {'name':thirdStage, 'params':thirdStageParams},
    }

In [14]:
def storeCompositionData(dir: Text, compositionHighLevelInfo: Dict):
    def storeFunction(stage): 
        compositionHighLevelInfoCopy = copy.deepcopy(compositionHighLevelInfo)
        name = stage.name
        if name == compositionHighLevelInfoCopy['firstStage']['name']:
            compositionHighLevelInfoCopy.pop('secondStage')
            compositionHighLevelInfoCopy.pop('thirdStage')
        elif name == compositionHighLevelInfoCopy['secondStage']['name']:
            compositionHighLevelInfoCopy.pop('thirdStage')

        md5 = hashlib.md5()
        dump = json.dumps(compositionHighLevelInfoCopy)
        md5.update(dump.encode())
        compositionHash = md5.hexdigest()

        compositionHighLevelInfoCopy['solution'] = [int(v) for v in stage.output['problemInstance'].solution]
        compositionHighLevelInfoCopy['objective'] = int(stage.output['problemInstance'].objective(isMinimizing = False))
        compositionHighLevelInfoCopy['cumulativeProcessTime'] = float(stage.cumulativeProcessTime)

        with open(f"{dir}/{compositionHash}.json", "w") as f:
            f.write(json.dumps(compositionHighLevelInfoCopy))

    return storeFunction

In [10]:
np.random.seed(2)
for i in range(100): 
    EXPERIMENT_PATH = f"{DATA_PATH}/exp_{i}"
    instancePath = f"{EXPERIMENT_PATH}/instance.txt"
    os.makedirs(EXPERIMENT_PATH, exist_ok = True)

    instance = generateKnapsackInstance(10,50,2,7,100,900,10,90,instancePath)
    
    COMPOSITION_PATH = f"{EXPERIMENT_PATH}/compositions"
    os.makedirs(COMPOSITION_PATH, exist_ok = True)
    
    for j in range(10):
        composition, stagesHighLevelInfo = getRandomThreeStagesPipeline(firstStages, secondStages, thirdStages)
        composition.getSolution(instance, storeCompositionData(COMPOSITION_PATH, stagesHighLevelInfo))

In [19]:
# from utils.instancesRepresentation import KnapsackInstance

# EXPERIMENT_PATH = f"{DATA_PATH}/exp-teste"
# instancePath = f"{EXPERIMENT_PATH}/instance.txt"
# instance = KnapsackInstance()
# instance.loadInstanceFromFile('./teste.txt')
# # instance.loadInstanceFromFile('../examples/knapsack/multipleBinaryKnapsack/knapsackExample.txt')

# COMPOSITION_PATH = f"{EXPERIMENT_PATH}/compositions"
# os.makedirs(COMPOSITION_PATH, exist_ok = True)

# for j in range(1):
#     composition, stagesHighLevelInfo = getRandomThreeStagesPipeline(firstStages, secondStages, thirdStages)
#     composition.getSolution(instance, storeCompositionData(COMPOSITION_PATH, stagesHighLevelInfo))

geneticAlgorithm -> tabuSearch -> firstImprovement
